# 5. Compare stock prices interactively using Bokeh

Data from https://www.kaggle.com/datasets/dgawlik/nyse

In [3]:
!pip install bokeh

     ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
     - -------------------------------------- 0.2/7.8 MB 6.9 MB/s eta 0:00:02
     - -------------------------------------- 0.4/7.8 MB 5.7 MB/s eta 0:00:02
     -- ------------------------------------- 0.5/7.8 MB 3.8 MB/s eta 0:00:02
     --- ------------------------------------ 0.7/7.8 MB 4.5 MB/s eta 0:00:02
     --- ------------------------------------ 0.7/7.8 MB 3.8 MB/s eta 0:00:02
     ----- ---------------------------------- 1.1/7.8 MB 4.1 MB/s eta 0:00:02
     ------- -------------------------------- 1.5/7.8 MB 4.7 MB/s eta 0:00:02
     -------- ------------------------------- 1.7/7.8 MB 4.7 MB/s eta 0:00:02
     ---------- ----------------------------- 2.1/7.8 MB 5.1 MB/s eta 0:00:02
     ---------- ----------------------------- 2.1/7.8 MB 4.7 MB/s eta 0:00:02
     ------------ --------------------------- 2.4/7.8 MB 4.9 MB/s eta 0:00:02
     -------------- ------------------------- 2.9/7.8 MB 5.3 MB/s eta 0

In [4]:
import pandas as pd# make bokeh display figures inside the notebook
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from ipywidgets import interact, widgets

# set bokeh to display within notebook
output_notebook()

Loading BokehJS ...

In [8]:
df = pd.read_csv('./data/nyse kaggle/prices.csv')
df.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [36]:
# map the date of each row to only the year-month-day format
from datetime import datetime

def shorten_time_stamp(timestamp):
    shortened = timestamp[0]
    
    if len(shortened) > 10:
        parsed_date=datetime.strptime(shortened, '%Y-%m-%d %H:%M:%S')
        shortened=datetime.strftime(parsed_date, '%Y-%m-%d')
    
    return shortened

df['short_date'] = df.apply(lambda x: shorten_time_stamp(x), axis=1)

df.head()

,date,symbol,open,close,low,high,volume,short_date
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0,2016-01-05
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0,2016-01-06
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0,2016-01-07
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0,2016-01-08
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0,2016-01-11


In [23]:
def add_candle_plot(plot, stock_name, stock_range, color):
    inc_1 = stock_range.close > stock_range.open
    dec_1 = stock_range.open > stock_range.close
    w = 0.5

    plot.segment(stock_range['short_date'], stock_range['high'], 
                 stock_range['short_date'], stock_range['low'], 
                 color="grey")

    plot.vbar(stock_range['short_date'][inc_1], w, 
              stock_range['high'][inc_1], stock_range['close'][inc_1], 
              fill_color="green", line_color="black",
              legend_label=('Mean price of ' + stock_name), muted_alpha=0.2)

    plot.vbar(stock_range['short_date'][dec_1], w, 
              stock_range['high'][dec_1], stock_range['close'][dec_1], 
              fill_color="red", line_color="black",
              legend_label=('Mean price of ' + stock_name), muted_alpha=0.2)

    stock_mean_val=stock_range[['high', 'low']].mean(axis=1)
    plot.line(stock_range['short_date'], stock_mean_val, 
              legend_label=('Mean price of ' + stock_name), muted_alpha=0.2,
              line_color=color, alpha=0.5)

In [32]:
# method to build the plot
def get_plot(stock_1, stock_2, date, value):    
    stock_1 = df[df['symbol'] == stock_1]
    stock_2 = df[df['symbol'] == stock_2]
    
    stock_1_name=stock_1['symbol'].unique()[0]
    stock_1_range=stock_1[(stock_1['short_date'] >= date[0]) & (stock_1['short_date'] <= date[1])]
    stock_2_name=stock_2['symbol'].unique()[0]
    stock_2_range=stock_2[(stock_2['short_date'] >= date[0]) & (stock_2['short_date'] <= date[1])]

    plot=figure(title='Stock prices', 
                     x_axis_label='Date', 
                     x_range=stock_1_range['short_date'], 
                     y_axis_label='Price in $USD',
                     width=800, 
                     height=500)
    
    plot.xaxis.major_label_orientation = 1
    plot.grid.grid_line_alpha=0.3
    
    if value == 'open-close':
        add_candle_plot(plot, stock_1_name, stock_1_range, 'blue')
        add_candle_plot(plot, stock_2_name, stock_2_range, 'orange')
        
    if value == 'volume':
        plot.line(stock_1_range['short_date'], stock_1_range['volume'], 
                  legend_label=stock_1_name, muted_alpha=0.2)
        plot.line(stock_2_range['short_date'], stock_2_range['volume'], 
                  legend_label=stock_2_name, muted_alpha=0.2,
                  line_color='orange')
    
    plot.legend.click_policy="mute"
    
    return plot


In [33]:
# extracing the necessary data
stock_names=df['symbol'].unique()
dates_2016=df[df['short_date'] >= '2016-01-01']['short_date']
unique_dates_2016=sorted(dates_2016.unique())
value_options=['open-close', 'volume']

In [34]:
# setting up the interaction elements
drp_1=widgets.Dropdown(options=stock_names,
                       value='AAPL',
                       description='Compare:')

drp_2=widgets.Dropdown(options=stock_names,
                       value='AON',
                       description='to:')

range_slider=widgets.SelectionRangeSlider(options=unique_dates_2016, 
                                          index=(0,25), 
                                          continuous_update=False,
                                          description='From-To',
                                          layout={'width': '500px'})

value_radio=widgets.RadioButtons(options=value_options,
                                 value='open-close',
                                 description='Metric')

In [35]:
# creating the interact method 
@interact(stock_1=drp_1, stock_2=drp_2, date=range_slider, value=value_radio)
def get_stock_for_2016(stock_1, stock_2, date, value):
    show(get_plot(stock_1, stock_2, date, value))

interactive(children=(Dropdown(description='Compare:', index=4, options=('WLTW', 'A', 'AAL', 'AAP', 'AAPL', 'A…